In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import altair as alt
alt.data_transformers.disable_max_rows()
import os
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
user_df = pd.read_csv("../data/entity_files/users_dataset.csv")
repo_df = pd.read_csv("../data/large_files/entity_files/repos_dataset.csv", low_memory=False)
search_queries_repo_join_df = pd.read_csv("../data/join_files/search_queries_repo_join_dataset.csv")
search_queries_user_join_df = pd.read_csv("../data/join_files/search_queries_user_join_dataset.csv")

# core_repos = repo_df[repo_df["id"].isin(search_queries_repo_join_df["id"].unique())]
# core_users = user_df[(user_df.login.isin(top_contributors.login)) | (user_df.login.isin(search_queries_user_join_df.login)) | (user_df.login.isin(core_repos['owner.login']))].drop_duplicates(subset=['login'])

### Determine Core Users

In [3]:
core_dfs = []

#### Contributors

In [4]:
contributors_df = pd.read_csv('../data/join_files/repo_contributors_join_dataset.csv')
contributor_counts = contributors_df.groupby(['login']).size().reset_index(name='counts')
freq_value = contributor_counts.describe().loc['75%'].values[0]
top_contributors = contributor_counts[contributor_counts.counts > freq_value]
top_contributors['type'] = 'contributor'
core_dfs.append(top_contributors)
print(len(top_contributors))

573


In [5]:
alt.Chart(contributor_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

alt.Chart(...)

#### Forks

In [6]:
forks_df = pd.read_csv('../data/join_files/repo_forks_join_dataset.csv')
forks_counts = forks_df.groupby(['owner.login']).size().reset_index(name='counts')
freq_value = forks_counts.describe().loc['75%'].values[0]
top_forks = forks_counts[forks_counts.counts > freq_value]
top_forks['type'] = 'fork'
top_forks = top_forks.rename(columns={'owner.login': 'login'})
core_dfs.append(top_forks)
print(len(top_forks))

168


In [7]:
alt.Chart(forks_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

alt.Chart(...)

#### Stargazers

In [8]:
stargazers_df = pd.read_csv('../data/join_files/repo_stargazers_join_dataset.csv')
stargazers_counts = stargazers_df.groupby(['user.login']).size().reset_index(name='counts')
freq_value = stargazers_counts.describe().loc['75%'].values[0]
top_stargazers = stargazers_counts[stargazers_counts.counts > freq_value]
top_stargazers['type'] = 'stargazer'
top_stargazers = top_stargazers.rename(columns={'user.login': 'login'})
core_dfs.append(top_stargazers)
print(len(top_stargazers))

764


In [9]:
alt.Chart(stargazers_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

alt.Chart(...)

#### Subscribers

In [10]:
subscribers_df = pd.read_csv('../data/join_files/repo_subscribers_join_dataset.csv')
subscribers_counts = subscribers_df.groupby(['login']).size().reset_index(name='counts')
freq_value = subscribers_counts.describe().loc['75%'].values[0]
top_subscribers = subscribers_counts[subscribers_counts.counts > freq_value]
top_subscribers['type'] = 'subscriber'
core_dfs.append(top_subscribers)
print(len(top_subscribers))

280


In [11]:
alt.Chart(subscribers_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

alt.Chart(...)

#### Issues

In [12]:
issues_df = pd.read_csv('../data/large_files/join_files/repo_issues_join_dataset.csv')
issues_counts = issues_df.groupby(['user.login']).size().reset_index(name='counts')
freq_value = issues_counts.describe().loc['75%'].values[0]
top_issues = issues_counts[issues_counts.counts > freq_value]
top_issues['type'] = 'issue'
top_issues = top_issues.rename(columns={'user.login': 'login'})
core_dfs.append(top_issues)
print(len(top_issues))

459


In [13]:
alt.Chart(issues_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

alt.Chart(...)

##### Issue Comments

In [14]:
issue_comments_df = pd.read_csv('../data/large_files/join_files/issues_comments_join_dataset.csv')
issue_comments_counts = issue_comments_df.groupby(['user.login']).size().reset_index(name='counts')
freq_value = issue_comments_counts.describe().loc['75%'].values[0]
top_issue_comments = issue_comments_counts[issue_comments_counts.counts > freq_value]
top_issue_comments['type'] = 'issue_comment'
top_issue_comments = top_issue_comments.rename(columns={'user.login': 'login'})
core_dfs.append(top_issue_comments)
print(len(top_issue_comments))

497


In [15]:
alt.Chart(issue_comments_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

alt.Chart(...)

#### Pulls

In [16]:
pulls_df = pd.read_csv('../data/large_files/join_files/repo_pulls_join_dataset.csv')
pulls_counts = pulls_df.groupby(['user.login']).size().reset_index(name='counts')
freq_value = pulls_counts.describe().loc['75%'].values[0]
top_pulls = pulls_counts[pulls_counts.counts > freq_value]
top_pulls['type'] = 'pull'
top_pulls = top_pulls.rename(columns={'user.login': 'login'})
core_dfs.append(top_pulls)
print(len(top_pulls))

201


In [17]:
alt.Chart(pulls_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

alt.Chart(...)

##### Pulls Comments

In [18]:
pull_comments_df = pd.read_csv('../data/large_files/join_files/pulls_comments_join_dataset.csv')
pull_comments_counts = pull_comments_df.groupby(['user.login']).size().reset_index(name='counts')
freq_value = pull_comments_counts.describe().loc['75%'].values[0]
top_pull_comments = pull_comments_counts[pull_comments_counts.counts > freq_value]
top_pull_comments['type'] = 'pull_comment'
top_pull_comments = top_pull_comments.rename(columns={'user.login': 'login'})
core_dfs.append(top_pull_comments)
print(len(top_pull_comments))

41


In [19]:
alt.Chart(pull_comments_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

alt.Chart(...)

#### User Followers

In [20]:
user_followers_df = pd.read_csv('../data/join_files/user_followers_join_dataset.csv')
user_followers_counts = user_followers_df.groupby(['login']).size().reset_index(name='counts')
freq_value = user_followers_counts.describe().loc['75%'].values[0]
top_user_followers = user_followers_counts[user_followers_counts.counts > freq_value]
top_user_followers['type'] = 'user_follower'
core_dfs.append(top_user_followers)
print(len(top_user_followers))

3399


In [21]:
alt.Chart(user_followers_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

alt.Chart(...)

#### User Following

In [22]:
user_following_df = pd.read_csv('../data/large_files/join_files/user_following_join_dataset.csv', low_memory=False)
user_following_counts = user_following_df.groupby(['login']).size().reset_index(name='counts')
freq_value = user_following_counts.describe().loc['75%'].values[0]
top_user_following = user_following_counts[user_following_counts.counts > freq_value]
top_user_following['type'] = 'user_following'
core_dfs.append(top_user_following)
print(len(top_user_following))

2986


In [23]:
alt.Chart(user_following_counts).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)

alt.Chart(...)

### Explore Top Users

In [24]:
top_users = pd.concat(core_dfs)

In [25]:
grouped_counts_top_users = top_users.groupby(['login']).size().reset_index(name='unique_counts').sort_values(by=['unique_counts'], ascending=False)
grouped_sum_top_users = top_users.groupby(['login'])['counts'].sum().reset_index(name='sum_counts').sort_values(by=['sum_counts'], ascending=False)
merged_top_users = pd.merge(grouped_sum_top_users, grouped_counts_top_users, on='login')

In [26]:
alt.Chart(merged_top_users).mark_circle().encode(
    alt.X('unique_counts'),
    alt.Y('sum_counts'),
    tooltip=['login', 'unique_counts', 'sum_counts']
)

alt.Chart(...)

In [27]:
total_interactions = merged_top_users.describe()['sum_counts'].loc['50%']
unique_interactions = merged_top_users.describe()['unique_counts'].loc['50%']

In [28]:
final_top_users = merged_top_users[(merged_top_users.sum_counts > total_interactions) & (merged_top_users.unique_counts > unique_interactions)]

In [29]:
print(f"From all our interactions we have identified {len(merged_top_users)} users that have interacted with the repositories in our dataset. From those we only want to keep the ones that are most consistent in interacting with DH repos. We have identified {len(final_top_users)} users that have interacted with repos more than {total_interactions} overall times and have interacted with more than {unique_interactions} different repositories.")

From all our interactions we have identified 6626 users that have interacted with the repositories in our dataset. From those we only want to keep the ones that are most consistent in interacting with DH repos. We have identified 1511 users that have interacted with repos more than 6.0 overall times and have interacted with more than 1.0 different repositories.


In [30]:
org_members_df = pd.read_csv('../data/join_files/org_members_dataset.csv')

In [31]:
members = user_df[user_df.login.isin(org_members_df.login)]
not_top_members = members[~members.login.isin(final_top_users.login)]
not_users = not_top_members[~not_top_members.login.isin(search_queries_user_join_df.login)]

In [32]:
print(f"From our list of {len(members)} members, {len(not_top_members)} are not in our top users list and {len(not_users)} are not in our search queries list. Which means that we have some users that are part of DH organizations but that don't interact much with DH repositories.")

From our list of 263 members, 147 are not in our top users list and 141 are not in our search queries list. Which means that we have some users that are part of DH organizations but that don't interact much with DH repositories.


In [33]:
core_repos = repo_df[repo_df["id"].isin(search_queries_repo_join_df["id"].unique())]

In [34]:
final_users = user_df[(user_df.login.isin(final_top_users.login)) | (user_df.login.isin(core_repos['owner.login'])) | (user_df.login.isin(search_queries_user_join_df.login))]

In [35]:
final_members = members[(members.login.isin(final_users.login)) ]

In [36]:
core_users = pd.concat([final_members, final_users])

In [37]:
core_users = core_users.drop_duplicates(subset=['login']).reset_index(drop=True)

In [38]:
allowed_bots = ['scottbot', 'ruebot', 'pyup-bot', 
       'doppins-bot', 'depfu[bot]', 'snyk-bot', 'navbot',
       'chaobotic-vasu']

In [39]:
core_users[~(core_users.login.str.contains('bot')) | (core_users.login.isin(allowed_bots))].to_csv('../data/derived_files/core_users.csv', index=False)

In [46]:
user_starred_df = pd.read_csv('../data/large_files/join_files/user_starred_join_dataset.csv', low_memory=False)

In [50]:
final_top_users[final_top_users.login == 'gaybro8777']

,login,sum_counts,unique_counts
1044,gaybro8777,17,3


In [54]:
stars_by_users = user_starred_df.groupby(['user_login']).size().reset_index(name='counts')

In [56]:
alt.Chart(stars_by_users).mark_bar().encode(
    alt.X('counts', bin=alt.Bin(maxbins=100)),
    y='count()'
)
    

alt.Chart(...)

In [61]:
freq_value = stars_by_users.describe().loc['75%'].values[0]
top_stars = stars_by_users[stars_by_users.counts > 2000]

In [62]:
top_stars

,user_login,counts
45,ByungjunKim,2190
135,JJ,2415
221,Pakillo,7071
284,ValRCS,2244
316,aculich,4476
317,ad-si,6078
411,ateucher,2631
462,brianzelip,6954
501,cheng10,3174
510,ck37,7761
